# Project 1 Introduction to Artificial Intelligence Andrea Buzzo

In this project, your Pacman agent will find paths through his maze world, both to reach a particular location and to collect food eﬀiciently

In this notebook there are all changes I do in this project

For work with this notebook we need to import all library and files

In [ ]:
# Library utili
import util 
# Librari heapq
import heapq

# We need to import all files 
from game import Actions
from typing import List, Tuple, Any
from game import Directions
from game import Agent
import time
import search
import pacman
import searchAgents

Now we need to import all classes from `Search.py` for working on first four questions

In [ ]:
class SearchProblem:
    """
    This class outlines the structure of a search problem, but doesn't implement
    any of the methods (in object-oriented terminology: an abstract class).

    You do not need to change anything in this class, ever.
    """
    def __init__(self, startingPosition, corners, walls):
        """
        Initializes the CornersProblem with the given starting position, corner
        coordinates, and wall positions.
        """
        self.startingPosition = startingPosition
        self.corners = corners
        self.walls = walls

    def getStartState(self):
        """
        Returns the start state for the search problem.
        """
        return (self.startingPosition, tuple(self.corners))

    def isGoalState(self, state):
        """
        Returns True if and only if the state is a valid goal state.
        """
        return not state[1]  # Check if there are no remaining corners

    def getSuccessors(self, state):
        """
        For a given state, this should return a list of triples, (successor,
        action, stepCost), where 'successor' is a successor to the current
        state, 'action' is the action required to get there, and 'stepCost' is
        the incremental cost of expanding to that successor.
        """
        successors = []
        pacman_position, remaining_corners = state

        for action in ['North', 'South', 'East', 'West']:
            x, y = pacman_position
            dx, dy = Actions.directionToVector(action)
            next_x, next_y = int(x + dx), int(y + dy)

            if not self.walls[next_x][next_y]:
                next_position = (next_x, next_y)
                next_corners = tuple([c for c in remaining_corners if c != next_position])
                successors.append(((next_position, next_corners), action, 1))  # Assuming constant cost

        return successors

    def getCostOfActions(self, actions):
        """
         actions: A list of actions to take

        This method returns the total cost of a particular sequence of actions.
        The sequence must be composed of legal moves.
        """
        return len(actions)


def tinyMazeSearch(problem):
    """
    Returns a sequence of moves that solves tinyMaze.  For any other maze, the
    sequence of moves will be incorrect, so only use this for tinyMaze.
    """
    from game import Directions
    s = Directions.SOUTH
    w = Directions.WEST
    return  [s, s, w, s, w, w, s, w]

Now we are prepared for work and complete the question

---

## Question 1: Finding a Fixed Food Dot using Depth First Search

In this question we need to find a path with using `Depth First Search Algorithm` and here there are my implementation for this algortihm, I start to implement this from the `DFS` pseudocode 

In [ ]:
def depthFirstSearch(problem: SearchProblem):
    """
    Search the deepest nodes in the search tree first.

    Your search algorithm needs to return a list of actions that reaches the
    goal. Make sure to implement a graph search algorithm.

    To get started, you might want to try some of these simple commands to
    understand the search problem that is being passed in:

    print("Start:", problem.getStartState())
    print("Is the start a goal?", problem.isGoalState(problem.getStartState()))
    print("Start's successors:", problem.getSuccessors(problem.getStartState()))
    """
    visited = set()  # To keep track of visited states
    stack = util.Stack()  # Stack for DFS

    # Push the start state onto the stack with an empty list of actions
    stack.push((problem.getStartState(), []))

    while not stack.isEmpty():
        state, actions = stack.pop()

        if problem.isGoalState(state):
            return actions  # Return the list of actions if the goal state is reached

        if state not in visited:
            visited.add(state)

            # Get successors and push them onto the stack with updated actions
            for successor, action, _ in problem.getSuccessors(state):
                new_actions = actions + [action]
                stack.push((successor, new_actions))

    # If no solution is found
    return []

For test this code we need to run this

In [ ]:
print("-----------------------------------------------------------")
!python3 pacman.py -l tinyMaze -p SearchAgent
print("-----------------------------------------------------------")
!python3 pacman.py -l mediumMaze -p SearchAgent 
print("-----------------------------------------------------------")
!python3 pacman.py -l bigMaze -z .5 -p SearchAgent
print("-----------------------------------------------------------")

---

## Question 2: Breadth First Search

In this question we need to find a path with using `Breadth First Search Algorithm` and here there are my implementation for this algortihm, I start to implement this from the `BFS` pseudocode 

In [ ]:
def breadthFirstSearch(problem: SearchProblem):
    """
    Search the shallowest nodes in the search tree first.
    """
    visited = set()  # To keep track of visited states
    queue = util.Queue()  # Queue for BFS

    # Enqueue the start state onto the queue with an empty list of actions
    queue.push((problem.getStartState(), []))

    while not queue.isEmpty():
        state, actions = queue.pop()

        if problem.isGoalState(state):
            return actions  # Return the list of actions if the goal state is reached

        if state not in visited:
            visited.add(state)

            # Get successors and enqueue them onto the queue with updated actions
            for successor, action, _ in problem.getSuccessors(state):
                new_actions = actions + [action]
                queue.push((successor, new_actions))

    # If no solution is found
    return []

For test this code we need to run this

In [ ]:
print("-----------------------------------------------------------")
!python3 pacman.py -l mediumMaze -p SearchAgent -a fn=bfs
print("-----------------------------------------------------------")
!python3 pacman.py -l bigMaze -p SearchAgent -a fn=bfs -z .5
print("-----------------------------------------------------------")

---

## Question 3: Varying the Cost Function

In this question we need to find a path with using `Uniform Cost Search Algorithm` and here there are my implementation for this algortihm, I start to implement this from the `UCS` pseudocode 

In [ ]:
def uniformCostSearch(problem: SearchProblem):
    """
    Search the node of least total cost first.
    """
    visited = set()  # To keep track of visited states
    priority_queue = util.PriorityQueue()  # PriorityQueue for UCS

    # Enqueue the start state onto the priority queue with an initial cost of 0 and an empty list of actions
    priority_queue.push((problem.getStartState(), [], 0), 0)

    while not priority_queue.isEmpty():
        state, actions, cost = priority_queue.pop()

        if problem.isGoalState(state):
            return actions  # Return the list of actions if the goal state is reached

        if state not in visited:
            visited.add(state)

            # Get successors and enqueue them onto the priority queue with updated actions and cost
            for successor, action, step_cost in problem.getSuccessors(state):
                new_actions = actions + [action]
                new_cost = cost + step_cost
                priority_queue.push((successor, new_actions, new_cost), new_cost)

    # If no solution is found
    return []

For test this code we need to run this

In [ ]:
print("-----------------------------------------------------------")
!python3 pacman.py -l mediumMaze -p SearchAgent -a fn=ucs
print("-----------------------------------------------------------")
!python3 pacman.py -l mediumDottedMaze -p StayEastSearchAgent 
print("-----------------------------------------------------------")
!python3 pacman.py -l mediumScaryMaze -p StayWestSearchAgent
print("-----------------------------------------------------------")

----

## Question 4: $A^*$ Search

In this question we need to find a path with using `A* Search` and here there are my implementation for this algortihm.

In [ ]:
class Node:
    def __init__(self, state, actions, cost, heuristic):
        self.state = state
        self.actions = actions
        self.cost = cost
        self.heuristic = heuristic

    def __lt__(self, other):
        return (self.cost + self.heuristic) < (other.cost + other.heuristic)

def nullHeuristic(state, problem=None):
    """
    A heuristic function estimates the cost from the current state to the nearest
    goal in the provided SearchProblem.  This heuristic is trivial.
    """
    return 0

def aStarSearch(problem: SearchProblem, heuristic=nullHeuristic):
    """Search the node that has the lowest combined cost and heuristic first."""
    start_state = problem.getStartState()
    start_node = Node(state=start_state, actions=[], cost=0, heuristic=heuristic(start_state, problem))

    open_set = [start_node]
    closed_set = set()

    while open_set:
        current_node = heapq.heappop(open_set)

        if problem.isGoalState(current_node.state):
            return current_node.actions

        state_key = tuple(current_node.state)  # Convert set to tuple
        if state_key not in closed_set:
            closed_set.add(state_key)
            successors = problem.getSuccessors(current_node.state)

            for successor, action, step_cost in successors:
                new_actions = current_node.actions + [action]
                new_cost = current_node.cost + step_cost
                new_heuristic = heuristic(successor, problem)
                new_node = Node(state=successor, actions=new_actions, cost=new_cost, heuristic=new_heuristic)

                heapq.heappush(open_set, new_node)

    return None

For test this code we need to run this

In [ ]:
print("-----------------------------------------------------------")
!python3 pacman.py -l bigMaze -z .5 -p SearchAgent -a fn=astar,heuristic=manhattanHeuristic
print("-----------------------------------------------------------")

----

For Next question we need to set what is in the `searchAgent.py` 

In [ ]:
class GoWestAgent(Agent):
    "An agent that goes West until it can't."

    def getAction(self, state):
        "The agent receives a GameState (defined in pacman.py)."
        if Directions.WEST in state.getLegalPacmanActions():
            return Directions.WEST
        else:
            return Directions.STOP

class SearchAgent(Agent):
    """
    This very general search agent finds a path using a supplied search
    algorithm for a supplied search problem, then returns actions to follow that
    path.

    As a default, this agent runs DFS on a PositionSearchProblem to find
    location (1,1)

    Options for fn include:
      depthFirstSearch or dfs
      breadthFirstSearch or bfs


    Note: You should NOT change any code in SearchAgent
    """

    def __init__(self, fn='depthFirstSearch', prob='PositionSearchProblem', heuristic='nullHeuristic'):
        # Warning: some advanced Python magic is employed below to find the right functions and problems

        # Get the search function from the name and heuristic
        if fn not in dir(search):
            raise AttributeError(fn + ' is not a search function in search.py.')
        func = getattr(search, fn)
        if 'heuristic' not in func.__code__.co_varnames:
            print('[SearchAgent] using function ' + fn)
            self.searchFunction = func
        else:
            if heuristic in globals().keys():
                heur = globals()[heuristic]
            elif heuristic in dir(search):
                heur = getattr(search, heuristic)
            else:
                raise AttributeError(heuristic + ' is not a function in searchAgents.py or search.py.')
            print('[SearchAgent] using function %s and heuristic %s' % (fn, heuristic))
            # Note: this bit of Python trickery combines the search algorithm and the heuristic
            self.searchFunction = lambda x: func(x, heuristic=heur)

        # Get the search problem type from the name
        if prob not in globals().keys() or not prob.endswith('Problem'):
            raise AttributeError(prob + ' is not a search problem type in SearchAgents.py.')
        self.searchType = globals()[prob]
        print('[SearchAgent] using problem type ' + prob)

    def registerInitialState(self, state):
        """
        This is the first time that the agent sees the layout of the game
        board. Here, we choose a path to the goal. In this phase, the agent
        should compute the path to the goal and store it in a local variable.
        All of the work is done in this method!

        state: a GameState object (pacman.py)
        """
        if self.searchFunction == None: raise Exception("No search function provided for SearchAgent")
        starttime = time.time()
        problem = self.searchType(state) # Makes a new search problem
        self.actions  = self.searchFunction(problem) # Find a path
        if self.actions == None:
            self.actions = []
        totalCost = problem.getCostOfActions(self.actions)
        print('Path found with total cost of %d in %.1f seconds' % (totalCost, time.time() - starttime))
        if '_expanded' in dir(problem): print('Search nodes expanded: %d' % problem._expanded)

    def getAction(self, state):
        """
        Returns the next action in the path chosen earlier (in
        registerInitialState).  Return Directions.STOP if there is no further
        action to take.

        state: a GameState object (pacman.py)
        """
        if 'actionIndex' not in dir(self): self.actionIndex = 0
        i = self.actionIndex
        self.actionIndex += 1
        if i < len(self.actions):
            return self.actions[i]
        else:
            return Directions.STOP

class PositionSearchProblem(search.SearchProblem):
    """
    A search problem defines the state space, start state, goal test, successor
    function and cost function.  This search problem can be used to find paths
    to a particular point on the pacman board.

    The state space consists of (x,y) positions in a pacman game.

    Note: this search problem is fully specified; you should NOT change it.
    """

    def __init__(self, gameState, costFn = lambda x: 1, goal=(1,1), start=None, warn=True, visualize=True):
        """
        Stores the start and goal.

        gameState: A GameState object (pacman.py)
        costFn: A function from a search state (tuple) to a non-negative number
        goal: A position in the gameState
        """
        self.walls = gameState.getWalls()
        self.startState = gameState.getPacmanPosition()
        if start != None: self.startState = start
        self.goal = goal
        self.costFn = costFn
        self.visualize = visualize
        if warn and (gameState.getNumFood() != 1 or not gameState.hasFood(*goal)):
            print('Warning: this does not look like a regular search maze')

        # For display purposes
        self._visited, self._visitedlist, self._expanded = {}, [], 0 # DO NOT CHANGE

    def getStartState(self):
        return self.startState

    def isGoalState(self, state):
        isGoal = state == self.goal

        # For display purposes only
        if isGoal and self.visualize:
            self._visitedlist.append(state)
            import __main__
            if '_display' in dir(__main__):
                if 'drawExpandedCells' in dir(__main__._display): #@UndefinedVariable
                    __main__._display.drawExpandedCells(self._visitedlist) #@UndefinedVariable

        return isGoal

    def getSuccessors(self, state):
        """
        Returns successor states, the actions they require, and a cost of 1.

         As noted in search.py:
             For a given state, this should return a list of triples,
         (successor, action, stepCost), where 'successor' is a
         successor to the current state, 'action' is the action
         required to get there, and 'stepCost' is the incremental
         cost of expanding to that successor
        """

        successors = []
        for action in [Directions.NORTH, Directions.SOUTH, Directions.EAST, Directions.WEST]:
            x,y = state
            dx, dy = Actions.directionToVector(action)
            nextx, nexty = int(x + dx), int(y + dy)
            if not self.walls[nextx][nexty]:
                nextState = (nextx, nexty)
                cost = self.costFn(nextState)
                successors.append( ( nextState, action, cost) )

        # Bookkeeping for display purposes
        self._expanded += 1 # DO NOT CHANGE
        if state not in self._visited:
            self._visited[state] = True
            self._visitedlist.append(state)

        return successors

    def getCostOfActions(self, actions):
        """
        Returns the cost of a particular sequence of actions. If those actions
        include an illegal move, return 999999.
        """
        if actions == None: return 999999
        x,y= self.getStartState()
        cost = 0
        for action in actions:
            # Check figure out the next state and see whether its' legal
            dx, dy = Actions.directionToVector(action)
            x, y = int(x + dx), int(y + dy)
            if self.walls[x][y]: return 999999
            cost += self.costFn((x,y))
        return cost

class StayEastSearchAgent(SearchAgent):
    """
    An agent for position search with a cost function that penalizes being in
    positions on the West side of the board.

    The cost function for stepping into a position (x,y) is 1/2^x.
    """
    def __init__(self):
        self.searchFunction = search.uniformCostSearch
        costFn = lambda pos: .5 ** pos[0]
        self.searchType = lambda state: PositionSearchProblem(state, costFn, (1, 1), None, False)

class StayWestSearchAgent(SearchAgent):
    """
    An agent for position search with a cost function that penalizes being in
    positions on the East side of the board.

    The cost function for stepping into a position (x,y) is 2^x.
    """
    def __init__(self):
        self.searchFunction = search.uniformCostSearch
        costFn = lambda pos: 2 ** pos[0]
        self.searchType = lambda state: PositionSearchProblem(state, costFn)

def manhattanHeuristic(position, problem, info={}):
    "The Manhattan distance heuristic for a PositionSearchProblem"
    xy1 = position
    xy2 = problem.goal
    return abs(xy1[0] - xy2[0]) + abs(xy1[1] - xy2[1])

def euclideanHeuristic(position, problem, info={}):
    "The Euclidean distance heuristic for a PositionSearchProblem"
    xy1 = position
    xy2 = problem.goal
    return ( (xy1[0] - xy2[0]) ** 2 + (xy1[1] - xy2[1]) ** 2 ) ** 0.5

class FoodSearchProblem:
    """
    A search problem associated with finding the a path that collects all of the
    food (dots) in a Pacman game.

    A search state in this problem is a tuple ( pacmanPosition, foodGrid ) where
      pacmanPosition: a tuple (x,y) of integers specifying Pacman's position
      foodGrid:       a Grid (see game.py) of either True or False, specifying remaining food
    """
    def __init__(self, startingGameState: pacman.GameState):
        self.start = (startingGameState.getPacmanPosition(), startingGameState.getFood())
        self.walls = startingGameState.getWalls()
        self.startingGameState = startingGameState
        self._expanded = 0 # DO NOT CHANGE
        self.heuristicInfo = {} # A dictionary for the heuristic to store information

    def getStartState(self):
        return self.start

    def isGoalState(self, state):
        return state[1].count() == 0

    def getSuccessors(self, state):
        "Returns successor states, the actions they require, and a cost of 1."
        successors = []
        self._expanded += 1 # DO NOT CHANGE
        for direction in [Directions.NORTH, Directions.SOUTH, Directions.EAST, Directions.WEST]:
            x,y = state[0]
            dx, dy = Actions.directionToVector(direction)
            nextx, nexty = int(x + dx), int(y + dy)
            if not self.walls[nextx][nexty]:
                nextFood = state[1].copy()
                nextFood[nextx][nexty] = False
                successors.append( ( ((nextx, nexty), nextFood), direction, 1) )
        return successors

    def getCostOfActions(self, actions):
        """Returns the cost of a particular sequence of actions.  If those actions
        include an illegal move, return 999999"""
        x,y= self.getStartState()[0]
        cost = 0
        for action in actions:
            # figure out the next state and see whether it's legal
            dx, dy = Actions.directionToVector(action)
            x, y = int(x + dx), int(y + dy)
            if self.walls[x][y]:
                return 999999
            cost += 1
        return cost
    
def mazeDistance(point1: Tuple[int, int], point2: Tuple[int, int], gameState: pacman.GameState) -> int:
    """
    Returns the maze distance between any two points, using the search functions
    you have already built. The gameState can be any game state -- Pacman's
    position in that state is ignored.

    Example usage: mazeDistance( (2,4), (5,6), gameState)

    This might be a useful helper function for your ApproximateSearchAgent.
    """
    x1, y1 = point1
    x2, y2 = point2
    walls = gameState.getWalls()
    assert not walls[x1][y1], 'point1 is a wall: ' + str(point1)
    assert not walls[x2][y2], 'point2 is a wall: ' + str(point2)
    prob = PositionSearchProblem(gameState, start=point1, goal=point2, warn=False, visualize=False)
    return len(search.bfs(prob))

class AnyFoodSearchProblem(PositionSearchProblem):
    """
    A search problem for finding a path to any food.

    This search problem is just like the PositionSearchProblem, but has a
    different goal test, which you need to fill in below.  The state space and
    successor function do not need to be changed.

    The class definition above, AnyFoodSearchProblem(PositionSearchProblem),
    inherits the methods of the PositionSearchProblem.

    You can use this search problem to help you fill in the findPathToClosestDot
    method.
    """

    def __init__(self, gameState):
        "Stores information from the gameState.  You don't need to change this."
        # Store the food for later reference
        self.food = gameState.getFood()

        # Store info for the PositionSearchProblem (no need to change this)
        self.walls = gameState.getWalls()
        self.startState = gameState.getPacmanPosition()
        self.costFn = lambda x: 1
        self._visited, self._visitedlist, self._expanded = {}, [], 0 # DO NOT CHANGE

    def isGoalState(self, state: Tuple[int, int]):
        """
        The state is Pacman's position. Fill this in with a goal test that will
        complete the problem definition.
        """
        x, y = state
        return self.food[x][y]  # True if there is food at Pacman's position, else False

----

Now we are ready for the next four questions

## Question 5: Finding All the Corners

The real power of A* will only be apparent with a more challenging search problem, my implementation for finding all the corners are there

In [ ]:
class CornersProblem(search.SearchProblem):
    def __init__(self, startingGameState: pacman.GameState):
        self.walls = startingGameState.getWalls()
        self.startingPosition = startingGameState.getPacmanPosition()
        top, right = self.walls.height - 2, self.walls.width - 2
        self.corners = ((1, 1), (1, top), (right, 1), (right, top))
        for corner in self.corners:
            if not startingGameState.hasFood(*corner):
                print('Warning: no food in corner ' + str(corner))
        self._expanded = 0  # DO NOT CHANGE; Number of search nodes expanded

    def getStartState(self):
        return (self.startingPosition, self.corners)

    def isGoalState(self, state):
        pacman_position, remaining_corners = state
        return len(remaining_corners) == 0

    def getSuccessors(self, state):
        successors = []
        pacman_position, remaining_corners = state
        for action in [Directions.NORTH, Directions.SOUTH, Directions.EAST, Directions.WEST]:
            dx, dy = Actions.directionToVector(action)
            next_x, next_y = int(pacman_position[0] + dx), int(pacman_position[1] + dy)
            if not self.walls[next_x][next_y]:
                next_position = (next_x, next_y)
                next_remaining_corners = tuple(
                    corner for corner in remaining_corners if corner != next_position)
                successors.append(((next_position, next_remaining_corners), action, 0.5))

        self._expanded += 1  # DO NOT CHANGE
        return successors

For test this code we need to run this

In [ ]:
print("-----------------------------------------------------------")
!python3 pacman.py -l tinyCorners -p SearchAgent -a fn=bfs,prob=CornersProblem
print("-----------------------------------------------------------")
!python3 pacman.py -l mediumCorners -p SearchAgent -a fn=bfs,prob=CornersProblem
print("-----------------------------------------------------------")

----

## Question 6: Corners Problem: Heuristic

Implement a non-trivial, consistent heuristic for the CornersProblem in cornersHeuristic

In [ ]:
def cornersHeuristic(state, problem: CornersProblem):
    """
    A heuristic for the CornersProblem that calculates the Manhattan distance
    from the current state to the nearest remaining corner.

    state:   The current search state
             (a data structure you chose in your search problem)

    problem: The CornersProblem instance for this layout.

    This function should always return a number that is a lower bound on the
    shortest path from the state to a goal of the problem; i.e. it should be
    admissible (as well as consistent)."""
    
    corners = problem.corners
    walls = problem.walls

    pacman_position, remaining_corners = state

    if not remaining_corners:
        return 0  # If there are no remaining corners, the heuristic is 0

    total_distance = 0

    for corner in remaining_corners:
        # Calculate the Manhattan distance from the current position to the corner
        distance = abs(pacman_position[0] - corner[0]) + abs(pacman_position[1] - corner[1])
        total_distance += distance

    # Calculate the weighted average distance by dividing the total distance by the number of remaining corners
    heuristic_value = total_distance / len(remaining_corners)

    return heuristic_value


class AStarCornersAgent(SearchAgent):
    "A SearchAgent for FoodSearchProblem using A* and your foodHeuristic"
    def __init__(self):
        self.searchFunction = lambda prob: search.aStarSearch(prob, cornersHeuristic)
        self.searchType = CornersProblem

For test this code we need to run this

In [ ]:
print("-----------------------------------------------------------")
!python3 pacman.py -l mediumCorners -p AStarCornersAgent -z 0.5
print("-----------------------------------------------------------")

----

## Question 7: Eating All The Dots

Now we’ll solve a hard search problem: eating all the Pacman food in as few steps as possible

In [ ]:
def foodHeuristic(state: Tuple[Tuple, List[List]], problem: FoodSearchProblem):
    """
    Your heuristic for the FoodSearchProblem goes here.

    This heuristic must be consistent to ensure correctness.  First, try to come
    up with an admissible heuristic; almost all admissible heuristics will be
    consistent as well.

    If using A* ever finds a solution that is worse uniform cost search finds,
    your heuristic is *not* consistent, and probably not admissible!  On the
    other hand, inadmissible or inconsistent heuristics may find optimal
    solutions, so be careful.

    The state is a tuple ( pacmanPosition, foodGrid ) where foodGrid is a Grid
    (see game.py) of either True or False. You can call foodGrid.asList() to get
    a list of food coordinates instead.

    If you want access to info like walls, capsules, etc., you can query the
    problem.  For example, problem.walls gives you a Grid of where the walls
    are.

    If you want to *store* information to be reused in other calls to the
    heuristic, there is a dictionary called problem.heuristicInfo that you can
    use. For example, if you only want to count the walls once and store that
    value, try: problem.heuristicInfo['wallCount'] = problem.walls.count()
    Subsequent calls to this heuristic can access
    problem.heuristicInfo['wallCount']
    """
    pacman_position, food_grid = state
    foods = food_grid.asList()
    if not foods:
        return 0

    max_distance = 0
    for food in foods:
        key = pacman_position + food
        if key in problem.heuristicInfo:
            distance = problem.heuristicInfo[key]
        else:
            # Use manhattan distance can get 6/7
            distance = mazeDistance(pacman_position, food, problem.startingGameState)
            problem.heuristicInfo[key] = distance

        if distance > max_distance:
            max_distance = distance

    return max_distance

For test this code we need to run this

In [ ]:
print("-----------------------------------------------------------")
!python3 pacman.py -l testSearch -p AStarFoodSearchAgent
print("-----------------------------------------------------------")
!python3 pacman.py -l trickySearch -p AStarFoodSearchAgent
print("-----------------------------------------------------------")

---

## Question 8: Suboptimal Search

Sometimes, even with A* and a good heuristic, finding the optimal path through all the dots is hard

In [ ]:
class ClosestDotSearchAgent(SearchAgent):
    "Search for all food using a sequence of searches"
    def registerInitialState(self, state):
        self.actions = []
        currentState = state
        while(currentState.getFood().count() > 0):
            nextPathSegment = self.findPathToClosestDot(currentState) # The missing piece
            self.actions += nextPathSegment
            for action in nextPathSegment:
                legal = currentState.getLegalActions()
                if action not in legal:
                    t = (str(action), str(currentState))
                    raise Exception('findPathToClosestDot returned an illegal move: %s!\n%s' % t)
                currentState = currentState.generateSuccessor(0, action)
        self.actionIndex = 0
        print('Path found with cost %d.' % len(self.actions))

    def findPathToClosestDot(self, gameState: pacman.GameState):
        """
        Returns a path (a list of actions) to the closest dot, starting from
        gameState.
        """
        # Here are some useful elements of the startState
        startPosition = gameState.getPacmanPosition()
        food = gameState.getFood()
        walls = gameState.getWalls()
        problem = AnyFoodSearchProblem(gameState)

        # Path cost per algorithm :
        return search.bfs(problem)   # 350
        #return search.dfs(problem)   # 5324
        #return search.ucs(problem)   # 350
        #return search.astar(problem) # 350
        # I choose bfs because is more simple

For test this code we need to run this

In [23]:
print("-----------------------------------------------------------")
!python3 pacman.py -l bigSearch -p ClosestDotSearchAgent -z .5
print("-----------------------------------------------------------")

-----------------------------------------------------------
[SearchAgent] using function depthFirstSearch
[SearchAgent] using problem type PositionSearchProblem
Path found with cost 350.
Pacman emerges victorious! Score: 2360
Average Score: 2360.0
Scores:        2360.0
Win Rate:      1/1 (1.00)
Record:        Win
-----------------------------------------------------------
